# COCO Test

In [1]:
# %pip install coco-experiment cocopp modcma numpy pandas

In [2]:
import cocoex  # experimentation module
import cocopp  # post-processing module (not strictly necessary)
import modcma  # ModCMA as solver
import numpy as np
import scipy.optimize

In [3]:
import warnings
# Filter out unwanted termination criteria warnings from modcma
# warnings.filterwarnings(action = "ignore", message = "Termination criteria met.*", module = "modcma")
warnings.filterwarnings('ignore')

In [74]:
# Schedule Setup

schedule_name = "GRS"
schedules = {}

schedules[2] = ['SLSQP', 'Powell', 'Powell', 'SLSQP', '2L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '2L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '2L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '2L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '2L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '2L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', '16L', 'Powell', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L', '16L']
schedules[3] = ['SLSQP', 'default', 'SLSQP', '4L', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'Powell', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'Powell', 'L-BFGS-B', '8L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '8L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '8L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '8L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '8L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '8L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '8L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'default', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell', '16L', '16L', 'Powell']
schedules[5] = ['SLSQP', 'default', 'SLSQP', '4L', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'Powell', 'L-BFGS-B', '16L', 'L-BFGS-B', 'L-BFGS-B', 'Powell', 'L-BFGS-B', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '32L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '32L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', '32L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', '32L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', '32L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '32L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '32L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', '32L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'L-BFGS-B', 'Powell', 'Powell', '32L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '32L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', '32L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', '32L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', '32L', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '4L', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell', '16L', 'Powell', 'Powell']
schedules[10] = ['SLSQP', 'default', 'SLSQP', 'SLSQP', '4L', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '8L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '16L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '16L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', 'L-BFGS-B', '2L', '32L', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', '2L', '32L', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', '2L', '32L', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', '2L', '32L', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', '2L', '32L', 'L-BFGS-B', 'L-BFGS-B', '32L', 'Powell', '2L', 'Powell', '32L', 'Powell', 'L-BFGS-B', 'Powell', '2L', '32L', 'Powell', 'Powell', 'L-BFGS-B', '32L', 'Powell', '2L', 'Powell', 'L-BFGS-B', '32L', 'Powell', 'Powell', 'L-BFGS-B', '2L', '32L', 'Powell', 'Powell', 'L-BFGS-B', '32L', 'Powell', '2L', 'Powell', 'Powell', '32L', 'L-BFGS-B', 'Powell', '2L', 'Powell', '32L', 'Powell', 'L-BFGS-B', 'Powell', '32L', '2L', 'Powell', 'L-BFGS-B', 'Powell', '32L', '2L', 'Powell', 'Powell', '32L', 'L-BFGS-B', 'Powell', '2L', 'Powell', '32L', 'L-BFGS-B', 'Powell', '2L', 'Powell', '32L', 'Powell', 'L-BFGS-B', '32L', 'Powell', '2L', 'Powell', 'L-BFGS-B', '32L', 'Powell', '2L', 'Powell', '32L', 'Powell', 'L-BFGS-B', '2L', 'Powell', '32L', 'Powell', '2L', 'Powell', '32L', 'Powell', 'L-BFGS-B', '2L', 'Powell', '32L', 'Powell', '2L', 'Powell', '32L', 'Powell', 'L-BFGS-B', '2L', 'Powell', '32L', 'Powell', '2L', 'Powell', '32L', 'Powell', 'L-BFGS-B', '2L', 'Powell', '32L', 'Powell', '2L', 'Powell', '32L', '2L', 'Powell', 'L-BFGS-B', 'Powell', '32L', '2L', 'Powell', '2L', 'Powell', '32L', 'Powell', '2L', '32L', 'Powell', 'Powell', 'L-BFGS-B', '2L', '32L', 'Powell', '2L', 'Powell', '32L', 'Powell', '2L', 'Powell', '32L', 'Powell', '2L', 'Powell', '32L', '2L', 'Powell', '2L', 'Powell', '32L', 'Powell', '2L', '32L', 'Powell', 'L-BFGS-B', 'Powell', '2L', '32L', 'Powell', '2L', 'Powell', '32L', 'Powell', '2L', 'Powell', '2L', '32L', 'Powell', '2L', 'Powell', '32L', 'Powell', '2L', '32L', 'Powell', '2L', 'Powell', '32L', 'Powell', '2L', 'Powell', '2L', '32L', 'Powell', '2L', 'Powell', '32L', 'Powell', '2L', 'Powell', '32L', '2L', 'Powell', '32L', 'Powell', '2L', 'Powell', '2L', '32L', 'Powell', 'Powell', '2L', '32L', 'Powell', '2L', 'Powell', '32L', '2L', 'Powell', '32L', 'Powell', '2L', 'Powell', '2L', '32L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', 'Powell', '64L', 'L-BFGS-B', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', 'L-BFGS-B', '2L', 'Powell', '64L', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', 'Powell', '64L', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'L-BFGS-B', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', 'Powell', '64L', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', 'Powell', '64L', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', 'L-BFGS-B', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '64L', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', 'Powell', '64L', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '64L', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', 'Powell', '64L', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', 'L-BFGS-B', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '64L', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '64L', 'L-BFGS-B', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '2L', '64L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '64L', '2L', 'Powell', '2L', '64L', 'Powell', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '64L', '2L', 'Powell', '64L', 'Powell', '2L', 'Powell', '64L', '2L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', '64L', 'Powell', 'Powell', 'L-BFGS-B', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L', '64L', 'Powell', '2L', 'Powell', '64L', 'Powell', '2L']

# schedule_name = "Powell"
# config_schedule = ["Powell"]

In [75]:
### prepare
suite_name = "bbob"
dimension = "2,3,5,10"

suite = cocoex.Suite(suite_name, "instances: 1-15", f"dimensions: {dimension}")  # see https://numbbo.github.io/coco-doc/C/#suite-parameters
output_folder = f'{schedule_name}'
observer = cocoex.Observer(suite_name, "result_folder: " + output_folder)

minimal_print = cocoex.utilities.MiniPrint()

COCO INFO: Results will be output to folder exdata/GRS


In [76]:
for problem in suite:  # solve all selected problems
    dim = problem.dimension
    budget = 100_000 * dim

    lambda_default = int(4 + np.floor(3 * np.log(dim))) # default

    cma_configurations = {
        "default": {},
        "2L": {"lambda_": 2 * lambda_default},
        "4L": {"lambda_": 4 * lambda_default},
        "8L": {"lambda_": 8 * lambda_default},
        "16L": {"lambda_": 16 * lambda_default},
        "32L": {"lambda_": 32 * lambda_default},
        "64L": {"lambda_": 64 * lambda_default},
        # "active": {"active": True},
        # "elitist": {"elitist": True},
        # "orthogonal": {"orthogonal": True},
        # "sequential": {"sequential": True},
        # "threshold_convergence": {"threshold_convergence": True},
        # "base_sampler_sobol": {"base_sampler": "sobol"},
        # "mirrored": {"mirrored": "mirrored"},
        # "mirrored_pairwise": {"mirrored": "mirrored pairwise"},
    }

    scipy_configurations = {
        "SLSQP": {"method": "SLSQP"},
        "L-BFGS-B": {"method": "L-BFGS-B"},
        "Powell": {"method": "Powell"},
    }

    config_schedule = schedules[problem.dimension]

    problem.observe_with(observer)  # generate data for cocopp
    problem(problem.dimension * [0])  # for better comparability

    restarts = 0

    np.random.seed(0xC0FFEE)

    # while not (problem.final_target_hit == 1 or problem.evaluations > budget):
    while not (problem.final_target_hit == 1 or problem.evaluations > 10 * budget):

        param_id = config_schedule[restarts % len(config_schedule)]
        if restarts % 1000 == 0 and restarts >= 1000:
            print(restarts)
        
        if param_id in cma_configurations.keys():
            params = cma_configurations[param_id]

            # cma = modcma.AskTellCMAES(dim, budget = budget, compute_termination_criteria = True, **params)
            cma = modcma.ModularCMAES(problem, x0 = problem.initial_solution_proposal(), d = dim, budget = budget, compute_termination_criteria = True, **params)

            while not (any(cma.break_conditions) or
                    any(cma.parameters.termination_criteria.values()) or
                    problem.final_target_hit == 1):        
                cma.step()
        
        else:
            params = scipy_configurations[param_id]
            scipy.optimize.minimize(problem, x0 = problem.initial_solution_proposal(), **params)
        
        restarts += 1
        observer.signal_restart(problem)

    minimal_print(problem)  # show progress

2D 17h15:21
 f1||||||||||||||| f2||||||||||||||| f3||||||||||||||| f4|||||||||||||||
 f5||||||||||||||| f6||||||||||||||| f7||||||||||||||| f8|||||||||||||||
 f9|||||||||||||||f10|||||||||||||||f11|||||||||||||||f12|||||||||||||||
f13|||||||||||||||f14|||||||||||||||f15|||||||||||||||f16|||||||||||||||
f17|||||||||||||||f18|||||||||||||||f19|||||||||||||||f20|||||||||||||||
f21|||||||||||||||f22|||||||||||||||f23|||||||||||||||f24|||||||||||||||
3D 17h15:59
 f1||||||||||||||| f2||||||||||||||| f3||||||||||||||| f4|||||||||||||||
 f5||||||||||||||| f6||||||||||||||| f7||||||||||||||| f8|||||||||||||||
 f9|||||||||||||||f10|||||||||||||||f11|||||||||||||||f12|||||||||||||||
f13|||||||||||||||f14|||||||||||||||f15|||||||||||||||f16|||||||||||||||
f17|||||||||||||||f18|||||||||||||||f19|||||||||||||||f20|||||||||||||||
f21|||||||||||||||f22|||||||||||||||f23|||||||||||||||f24||||||.||||||||
5D 17h17:44
 f1||||||||||||||| f2||||||||||||||| f3||||||||||||||| f4|||||||||||||||
 f5||||||||||||

In [27]:
restarts

2424

In [28]:
observer.result_folder

'exdata/L-BFGS-B'

In [79]:
### post-process data
cocopp.main(["--include-fonts", observer.result_folder, "portfolio-best", "bipop-cma!", "HCMA!"]);  # re-run folders look like "...-001" etc
# cocopp.main('exdata/Powell exdata/SLSQP exdata/L-BFGS-B exdata/default exdata/2L exdata/4L exdata/8L exdata/16L exdata/32L exdata/64L')

Post-processing (2+)
  Using 4 data sets:
    exdata/GRS
    portfolio-best
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/BIPOP-CMA-ES_hansen_noiseless.tgz
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2013/HCMA_loshchilov_noiseless.tgz

Post-processing (2+)
  loading data...
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/GRS
  Data consistent according to consistency_check() in pproc.DataSet
  using: portfolio-best
  using pickled DataSetList   using: /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/BIPOP-CMA-ES_hansen_noiseless.tgz
  using pickled DataSetList   using: /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2013/HCMA_loshchilov_noiseless.tgz
  Will generate output data in folder ppdata/GRS_portf_BIPOP_HCMA_2b22
    this might take several minutes.
ECDF graphs per noise group...
Loading best algorithm data from refalgs/best2009-bbob.tar.gz ...
  using pickled DataSetLi

## Performance Assessment

In [237]:
# datasets = cocopp.load(cocopp.archives.bbob.get_first("BIPOP"))
# datasets = cocopp.load(cocopp.archives.bbob.get_first("HCMA"))
datasets = cocopp.load(observer.result_folder)

  Data consistent according to consistency_check() in pproc.DataSet


In [239]:
np.mean([np.mean(np.clip(dset.detERT(np.power(10, np.linspace(2, -8, 51))), 0, 1_000_000 * dim)) for dset in dsets])

1084399.5905066065

## Create ideal portfolio algorithm

In [70]:
import cocopp.bestalg

individual_algos = ["exdata/Powell", "exdata/SLSQP", "exdata/L-BFGS-B", "exdata/default", "exdata/2L", "exdata/4L", "exdata/8L", "exdata/16L", "exdata/32L", "exdata/64L"]
cocopp.bestalg.custom_generate(individual_algos, algId = "portfolio-best")

In [80]:
cocopp.main(["--include-fonts", "portfolio-best", "exdata/GRS"] + individual_algos)

Post-processing (2+)
  Using 12 data sets:
    portfolio-best
    exdata/GRS
    exdata/Powell
    exdata/SLSQP
    exdata/L-BFGS-B
    exdata/default
    exdata/2L
    exdata/4L
    exdata/8L
    exdata/16L
    exdata/32L
    exdata/64L

Post-processing (2+)
  loading data...
  Data consistent according to consistency_check() in pproc.DataSet
  using: portfolio-best
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/GRS
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/Powell
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/SLSQP
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/L-BFGS-B
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/default
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/2L
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/4L
  

DictAlg([(('portfolio-best', ''),
          [DataSet(portfolio-best on f1 2-D),
           DataSet(portfolio-best on f2 2-D),
           DataSet(portfolio-best on f3 2-D),
           DataSet(portfolio-best on f4 2-D),
           DataSet(portfolio-best on f5 2-D),
           DataSet(portfolio-best on f6 2-D),
           DataSet(portfolio-best on f7 2-D),
           DataSet(portfolio-best on f8 2-D),
           DataSet(portfolio-best on f9 2-D),
           DataSet(portfolio-best on f10 2-D),
           DataSet(portfolio-best on f11 2-D),
           DataSet(portfolio-best on f12 2-D),
           DataSet(portfolio-best on f13 2-D),
           DataSet(portfolio-best on f14 2-D),
           DataSet(portfolio-best on f15 2-D),
           DataSet(portfolio-best on f16 2-D),
           DataSet(portfolio-best on f17 2-D),
           DataSet(portfolio-best on f18 2-D),
           DataSet(portfolio-best on f19 2-D),
           DataSet(portfolio-best on f20 2-D),
           DataSet(portfolio-best o

In [62]:
cocopp.main(["--include-fonts", "portfolio-best", "HCMA!", "bipop-cma!"])

Post-processing (2+)
  Using 3 data sets:
    portfolio-best
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2013/HCMA_loshchilov_noiseless.tgz
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/BIPOP-CMA-ES_hansen_noiseless.tgz

Post-processing (2+)
  loading data...
  Data consistent according to consistency_check() in pproc.DataSet
  using: portfolio-best
  using pickled DataSetList   using: /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2013/HCMA_loshchilov_noiseless.tgz
  using pickled DataSetList   using: /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/BIPOP-CMA-ES_hansen_noiseless.tgz
  Will generate output data in folder ppdata/portf_HCMA__BIPOP_b1ed
    this might take several minutes.
ECDF graphs per noise group...
Loading best algorithm data from refalgs/best2009-bbob.tar.gz ...
  using pickled DataSetList   using: /Users/lennartschapermeier/source/research/cma-rsmp/.conda/lib/python3.12/site-packages/cocopp

DictAlg([(('portfolio-best', ''),
          [DataSet(portfolio-best on f1 2-D),
           DataSet(portfolio-best on f2 2-D),
           DataSet(portfolio-best on f3 2-D),
           DataSet(portfolio-best on f4 2-D),
           DataSet(portfolio-best on f5 2-D),
           DataSet(portfolio-best on f6 2-D),
           DataSet(portfolio-best on f7 2-D),
           DataSet(portfolio-best on f8 2-D),
           DataSet(portfolio-best on f9 2-D),
           DataSet(portfolio-best on f10 2-D),
           DataSet(portfolio-best on f11 2-D),
           DataSet(portfolio-best on f12 2-D),
           DataSet(portfolio-best on f13 2-D),
           DataSet(portfolio-best on f14 2-D),
           DataSet(portfolio-best on f15 2-D),
           DataSet(portfolio-best on f16 2-D),
           DataSet(portfolio-best on f17 2-D),
           DataSet(portfolio-best on f18 2-D),
           DataSet(portfolio-best on f19 2-D),
           DataSet(portfolio-best on f20 2-D),
           DataSet(portfolio-best o

In [59]:
all_algs = [cocopp.archives.bbob.get_first(x) for x in cocopp.archives.bbob]

In [63]:
cocopp.main('bbob/2009/*')

Post-processing (1)
  Using 31 data sets:
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/ALPS_hornby_noiseless.tgz
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/AMALGAM_bosman_noiseless.tgz
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/BAYEDA_gallagher_noiseless.tgz
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/BFGS_ros_noiseless.tgz
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/BIPOP-CMA-ES_hansen_noiseless.tgz
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/CMA-ESPLUSSEL_auger_noiseless.tgz
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/Cauchy-EDA_posik_noiseless.tgz
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/DASA_korosec_noiseless.tgz
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/DE-PSO_garcia-nieto_noiseless.tgz
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da

DictAlg([(('ALPS_hornby', ''),
          [DataSet(ALPS_hornby on f1 2-D),
           DataSet(ALPS_hornby on f2 2-D),
           DataSet(ALPS_hornby on f3 2-D),
           DataSet(ALPS_hornby on f4 2-D),
           DataSet(ALPS_hornby on f5 2-D),
           DataSet(ALPS_hornby on f6 2-D),
           DataSet(ALPS_hornby on f7 2-D),
           DataSet(ALPS_hornby on f8 2-D),
           DataSet(ALPS_hornby on f9 2-D),
           DataSet(ALPS_hornby on f10 2-D),
           DataSet(ALPS_hornby on f11 2-D),
           DataSet(ALPS_hornby on f12 2-D),
           DataSet(ALPS_hornby on f13 2-D),
           DataSet(ALPS_hornby on f14 2-D),
           DataSet(ALPS_hornby on f15 2-D),
           DataSet(ALPS_hornby on f16 2-D),
           DataSet(ALPS_hornby on f17 2-D),
           DataSet(ALPS_hornby on f18 2-D),
           DataSet(ALPS_hornby on f19 2-D),
           DataSet(ALPS_hornby on f20 2-D),
           DataSet(ALPS_hornby on f21 2-D),
           DataSet(ALPS_hornby on f22 2-D),
          